In [ ]:
# Librerías
import pandas as pd
import numpy as np
from pathlib import Path
import os

import matplotlib.pyplot as plt
import seaborn as sns
from scipy import stats

import pickle

In [ ]:
sns.set_theme(style="darkgrid")

In [ ]:
# Variables
src_path = os.getcwd()
data_path = "/data/raw/"
p_data_path = "/data/processed/"
data_file = "merged_data.csv"

file_path= Path(os.path.dirname(src_path)+data_path+data_file)


In [ ]:
df_coffee = pd.read_csv(file_path)

In [ ]:
df_coffee.shape

In [ ]:
df_coffee.info()

<h2 style="text-align: center;">ANALIZANDO EL DATASET</h2>

El dataset consta de 1339 filas por 44 columnas y contiene la información relativa a diferentes variedades de café de tipo Arabica y Robusta calificadas por personal del CQI (Coffee Quality Institute).

La información se divide en:

Calidad de café (Cupping Score)
- Aroma
- Flavor
- Aftertaste
- Acidity
- Body
- Balance
- Uniformity
- Clean Cup
- Sweetness


Semilla (Bean)
- Species (arabica / robusta)
- Variety
- Processing Method
- Moisture
- Color
- Category.One.Defects
- Category.Two.Defects
- Quakers
- Expiration


Plantación (Farm)
- Country.of.Origin
- Harvest.Year
- Farm.Name
- Mill
- Owner
- Company
- Altitude + unit_of_measurement
- Region



La columna "Total Cup Points" será nuestro medidor de calidad, siendo mejor el café cuanto más alta sea esta puntuación.

Realizaremos un análisis univariante y multivariante de las información de la calidad y de la semilla junto con el país de origen, región, año de cosecha y altitud de la plantación con el objetivo final de encontrar un algoritmo que permita predecir qué café tendrá una mejor puntuación en función de todas sus características.

Para la información de la altitud, nos quedaremos con la columna "Altitude" y "altitude_mean_meters" que representa la media entre el valor máximo y mínimo de la altitud a la que se cultiva esa variedad de café en esa plantación. Además, necesitamos la columna "unit_of_measurement" para saber si la medida está en metros o pies.

<h2 style="text-align: center;">LIMPIEZA DEL DATASET</h2>

Borramos las columnas que no nos serán necesarias para la realización del análisis.

In [ ]:
df = df_coffee.drop(columns=["Unnamed: 0", "Lot.Number", "ICO.Number", "Producer", "Number.of.Bags",
                             "Bag.Weight", "In.Country.Partner", "Grading.Date", "Cupper.Points",
                             "Certification.Body", "Certification.Address", "Certification.Contact",
                             "altitude_low_meters", "altitude_high_meters", "Owner.1", "Farm.Name",
                             "Mill", "Company", "Owner"])

<h3 style="text-align: center;">Chequeo de duplicados</h3>

In [ ]:
# Cantidad de duplicados
df.duplicated().sum()

No hay registros duplicados.

<h3 style="text-align: center;">Chequeo de valores nulos</h3>

In [ ]:
# Null / Missing values
df[df.columns[df.isnull().any()]].isnull().sum().sort_values(ascending=False)

Hay bastante cantidad de valores nulos. Vamos a tratar de reducirlos todo lo posible.

<h4 style="text-align: center;"><ins>1 - Nulos en Region</ins></h4>

Saber la región en la que se cultiva un café puede ser importante para sus características.

Comprobamos si podemos inferir a qué región pertenece un café en función de su país de origen y la altitud media a la que se encuentra la plantación. La idea es que si uno de los registros nulos comparte altura y pais de origen con otro que que tenga un valor, supondremos que se trata de la misma región.

In [ ]:
df.loc[(df["Region"].isnull()) & (df["Country.of.Origin"].notnull()), ["Country.of.Origin", "Region", 'altitude_mean_meters']]

La práctica totalidad de los registros con valor nulo en el campo de Región también presenta valor nulo para su altura media, con lo que es imposible imputar valores y por tanto decidimos eliminar estos registros.

In [ ]:
df = df[df["Region"].notna()]

<h4 style="text-align: center;"><ins>2 - Nulos en altitud</ins></h4>


Uno de los valores que hemos considerado importante en el análisis es la altitud a la que se cultiva el café, por lo que es el primer dato que vamos a analizar 

1 - Comprobamos los valores de la columna "Altitud" que no son nulos junto con los de altitud media para tratar de imputar algún valor a esta última columna.

In [ ]:
df.loc[(df["altitude_mean_meters"].isnull()) & df["Altitude"].notnull(), ["Altitude", "altitude_mean_meters"]]

Con esos valores no podemos realizar ningún cálculo.

Antes de realizar las imputaciones debemos tener en cuenta la unidad de medida en que está la columna de altitud y asegurarnos que todos los valores están en metros.

In [ ]:
df["altitude_recalc"] = df.apply(lambda x: x["altitude_mean_meters"] if x["unit_of_measurement"]=='m' else x["altitude_mean_meters"]*0.3048, axis=1)

Estudiamos los valores estadísticos de la columna, pues si vamos a realizar imputaciones a la media o a la moda debemos asegurarnos de que no existan valores fuera de rango que nos provoquen errores.

In [ ]:
df["altitude_recalc"].describe()

Se observa un valor máximo demasiado elevado y un mínimo demasiado bajo, así como una desviación estándar alta. Cualquier valor por encima de los 3000m y por debajo de los 600m lo consideraremos un error y usaremos esos valores como extremos.

In [ ]:
df.loc[df["altitude_recalc"]>3000, "altitude_recalc"] = 3000
df.loc[df["altitude_recalc"]<600, "altitude_recalc"] = 600

In [ ]:
df["altitude_recalc"].describe()

Con unos valores más ajustados a la realidad, podemos pasar a tratar los valores nulos

2 - Imputaremos los valores de altitud correspondientes a la media de la región a la que pertenecen.

In [ ]:
df["altitude_recalc"].fillna(df.groupby(["Region"])["altitude_recalc"].transform("mean"), inplace=True)
print("Quedan {} valores nulos en la columna.". format(df["altitude_recalc"].isnull().sum()))

3 - Dado que aún quedan valores nulos, vamos a repetir el proceso pero imputando la moda de la altura con respecto al país. Decidimos no usar la media porque puede ocurrir que en un mismo país haya grandes diferencias en la altura en que sitúan las plantaciones.

In [ ]:
df["altitude_recalc"].fillna(df.groupby(['Country.of.Origin'])['altitude_recalc'].transform(lambda x: x.mode()[0]), inplace=True)
print("Quedan {} valores nulos en la columna.". format(df["altitude_recalc"].isnull().sum()))

Hemos quitado todos los valores nulos de la altitud media, por lo que la columna Altitude se hace innecesaria en el conjunto de datos y procedemos a eliminarla.

In [ ]:
df = df.drop(columns=["Altitude"])

<h4 style="text-align: center;"><ins>3 - Nulos en variedad</ins></h4>

Antes de estudiar los registros nulos comprobamos los diferentes tipos de variedad que hay en la tabla

In [ ]:
df["Variety"].unique()

Dado que ya existe un valor genérico establecido como "Other" decidimos imputarles a todos los nulos ese valor genérico.

In [ ]:
df["Variety"].fillna("Other", inplace = True)
print("Quedan {} valores nulos en la columna.". format(df["Variety"].isnull().sum()))

<h4 style="text-align: center;"><ins>4 - Nulos en Color</ins></h3>

Antes de estudiar los valores nulos, comprobamos los diferentes colores presentes en el dataset

In [ ]:
df["Color"].unique()

El color de las semillas de café, antes del tostado, suele depender de la región en la que se hayan cultivado, pero habitualmente son de color verde por lo que imputaremos a los valores nulos el color verde.

In [ ]:
df["Color"].fillna("Green", inplace=True)
print("Quedan {} valores nulos en la columna.". format(df["Color"].isnull().sum()))

<h4 style="text-align: center;"><ins>5 - Nulos en Método de procesado</ins></h3>

Como anteriormente, vamos a comprobar los tipos de métodos de procesado existentes en el dataframe.

In [ ]:
df["Processing.Method"].unique()

Dado que el método de procesado es difícil de imputar pues no tiene por qué seguir una lógica en función de la región o el país, vamos a imputar a todos los nulos el valor genérico "Other" que ya existe.

In [ ]:
df["Processing.Method"].fillna("Other", inplace=True)
print("Quedan {} valores nulos en la columna.". format(df["Processing.Method"].isnull().sum()))

<h4 style="text-align: center;"><ins>6 - Nulos en Año de cosecha</ins></h3>

In [ ]:
df["Harvest.Year"].unique()

Viendo la cantidad de variabilidad que hay en la columna, decidimos extraer la última aparición de una cifra de 4 dígitos y tomarlos como el año de la cosecha. Almacenaremos el resultado en una nueva columna.

In [ ]:
df["HarvestYear_Calc"] = df["Harvest.Year"].str.extract('(\d{4})(?!.*\d)')

Examinamos de nuevo los valores obtenidos tras la imputación.

In [ ]:
df["HarvestYear_Calc"].unique()

Nos quedan valores nulos, a los que imputaremos el valor genérico "unknown", pues para tratarlos deberíamos ir uno por uno.

In [ ]:
df["HarvestYear_Calc"].fillna("UNKNOWN", inplace=True)
print("Quedan {} valores nulos en la columna.". format(df["HarvestYear_Calc"].isnull().sum()))

Ya no es necesaria la columna original del año de cosecha, por lo que podemos eliminarla

In [ ]:
df.drop(columns=["Harvest.Year"], inplace=True)

<h4 style="text-align: center;"><ins>7 - Nulos en Quakers</ins></h3>

Los "Quakers" son aquéllas semillas de café que, por no estar maduras y por tanto no tener suficientes azúcares, no se caramelizan con el tostado y si no se retiran pueden provocar un empeoramiento de la calidad del café.

Esta columna indica el número de estas semillas encontradas en el saco de café.

In [ ]:
df[df["Quakers"].isnull()]

Como sólo tenemos un caso con un valor nulo y no podemos imputar ningún valor pues es una característica independiente a la región o al país, lo eliminaremos sin que afecte al desarrollo del análisis.

In [ ]:
df = df[df["Quakers"].notna()]

---

<h2 style="text-align: center;">ANÁLISIS DE LOS DATOS</h2>

<h4 style="text-align: center;"><ins>1 - País de Origen</ins></h4>

Examinamos los valores únicos de la columna, por si fuera necesario realizar alguna codificación o traducción en sus valores.

In [ ]:
df["Country.of.Origin"].unique()

Los valores son relativamente pocos y no es necesario realizar ningún cambio.

<h4 style="text-align: center;"><ins>2 - Especie</ins></h4>

Examinanos las especies presentes en el dataset.

In [ ]:
df["Species"].unique()

Solamente hay dos especies y ningún registro carece de esta información.

<h4 style="text-align: center;"><ins>3 - Variedad</ins></h4>

Examinamos los tipos de variedad presentes en el conjunto de datos.

In [ ]:
df["Variety"].unique()

Aunque son bastantes, no existe ningún valor extraño y todo aquéllo que no está definido tiene el valor genérico "Other"

<h4 style="text-align: center;"><ins>4 - Altitud</ins></h4>

Analizamos los valores de la columna relativa a la altitud media del cultivo. Dado que es un valor numérico, además del número de valores que presenta realizaremos un análisis de sus valores estadísticos.

In [ ]:
df["altitude_recalc"].describe()

Tener tanta variedad de alturas va a suponer un problema a la hora de analizar los datos y en el momento de las predicciones, por lo que se hace necesario realizar agrupamientos de valores a fin de simplificar esta categoría.

In [ ]:
df["altitude_groups"] = ["Below 1000m" if x<=1000 else ("Between 1000 and 2000m" if x>1000 and x<=2000 else "Above 2000m") for x in df["altitude_recalc"]]

<h4 style="text-align: center;"><ins>5 - Método de procesado</ins></h4>

Examinamos los valores existentes para el método de procesado.

In [ ]:
df["Processing.Method"].unique()

Todos los valores son correctos y existe un valor genérico.

<h4 style="text-align: center;"><ins>6 - Color</ins></h4>

Como en los casos anteriores, examinamos los valores del color que puede presentar la semilla del café.

In [ ]:
df["Color"].unique()

Al haber tratado con anterioridad los nulos, no es necesario realizar ninguna modificación adicional.

<h4 style="text-align: center;"><ins>7 - Quakers</ins></h4>

Listamos los valores que toma la columna "quakers"

In [ ]:
df["Quakers"].unique()

Examinaremos la cantidad de cada uno de los valores, por si fuera útil realizar agrupamientos.

In [ ]:
df["Quakers"].value_counts()

Ante esos valores, decidimos realizar una división en 3 grupos.

In [ ]:
df["quakers_groups"] = [str(x) if x>=0 and x<=2 else ("Between 3 and 5" if x>=3 and x<=5 else "More than 5") for x in df["Quakers"]]

<h4 style="text-align: center;"><ins>8 - Año de cosecha</ins></h4>

In [ ]:
df["HarvestYear_Calc"].unique()

In [ ]:
df["HarvestYear_Calc"].value_counts()

<h4 style="text-align: center;"><ins>9 - Variedad</ins></h4>

In [ ]:
df["Variety"].unique()

<h4 style="text-align: center;"><ins>10 - Aroma</ins></h4>

In [ ]:
df["Aroma"].unique()

In [ ]:
df["Aroma"].describe()

Los valores estadísticos de la puntuación de Aroma están dentro de los valores esperados, pero resulta difícil creer que exista una puntuación 0 para un valor, por lo que puede ser un error y debemos estudiarlo.

In [ ]:
df.loc[df["Aroma"]==0]

Puede observarse que tiene todos sus valores de calidad a 0 por lo que entendemos que es un error, ya sea porque esta variedad no se analizó o porque los registros de las puntuaciones no han sido almacenados.

En cualquier caso, vamos a eliminarlo pues no es de ninguna utilidad.

In [ ]:
df = df[df["Aroma"]>0]

<h4 style="text-align: center;"><ins>11 - Sabor</ins></h4>

In [ ]:
df["Flavor"].unique()

In [ ]:
df["Flavor"].describe()

Todos los valores estadísticos para la variable Flavor están dentro de los esperados y no parece haber ninguno fuera de rango.

<h4 style="text-align: center;"><ins>12 - Regusto</ins></h4>

In [ ]:
df["Aftertaste"].unique()

In [ ]:
df["Aftertaste"].describe()

Todos los valores estadísticos para la variable Aftertaste están dentro de los esperados y no parece haber ninguno fuera de rango.

<h4 style="text-align: center;"><ins>13 - Cuerpo</ins></h4>

In [ ]:
df["Body"].unique()

In [ ]:
df["Body"].describe()

Todos los valores estadísticos para la variable Body están dentro de los esperados y no parece haber ninguno fuera de rango.

<h4 style="text-align: center;"><ins>14 - Dulzor</ins></h4>

In [ ]:
df["Sweetness"].unique()

In [ ]:
df["Sweetness"].describe()

La variable dulzor presenta unos valores muy altos dentro del rango esperado; pero llama la atención la puntuación mínima de 1 y debemos estudiarla en profundidad. Vamos a extender el estudio a los valores por debajo de 5.

In [ ]:
df.loc[df["Sweetness"]<5, ["Species","Country.of.Origin","Region",	"Variety",	"Processing.Method", "Sweetness",	"Aroma",	"Flavor",	"Aftertaste",	"Acidity",	"Body",	"Quakers", "Clean.Cup"]]

Todo parece indicar que se trata de un error a la hora de capturar el dato, por lo que decidimos sustituir ese dato por la media de la puntuación de la variable Sweetness que presenta el país de origen.

In [ ]:
df.loc[(df["Country.of.Origin"]=="Guatemala"), "Sweetness"].mean()

Que la media de la variable sweetness del país sea tan grande nos refuerza la idea de que el valor 1.33 ha debido ser un error en la captura de datos.

In [ ]:
df.loc[df["Sweetness"]<2, "Sweetness"] = df.loc[(df["Country.of.Origin"]=="Guatemala"), "Sweetness"].mean()

In [ ]:
df["Sweetness"].describe()

Los valores ahora, pese a ser muy elevados, son más coherentes, sin valores extremos que podiéramos considerar como fuera de rango.

<h4 style="text-align: center;"><ins>15 - Acidez</ins></h4>

In [ ]:
df["Acidity"].unique()

In [ ]:
df["Acidity"].describe()

De nuevo tenemos una característica con los valores dentro de lo esperado, sin que se aprecie ninguno fuera de rango.

<h4 style="text-align: center;"><ins>16 - Equilibrio</ins></h4>

In [ ]:
df["Balance"].unique()

In [ ]:
df["Balance"].describe()

Los valores para la variable Balance están dentro de lo esperado, sin que se aprecie ninguno fuera de rango.

<h4 style="text-align: center;"><ins>17 - Uniformidad</ins></h4>

In [ ]:
df["Uniformity"].unique()

In [ ]:
df["Uniformity"].describe()

En los valores de la variable Uniformidad, pese a ser muy elevados, no se aprecian valores que podiéramos considerar como fuera de rango, aunque dado el valor de los percentiles parece que se agrupan casi todos en el máximo, lo que nos dará, presumiblemente, una distribución alejada de la normal.

<h4 style="text-align: center;"><ins>18 - Clean Cup</ins></h4>

In [ ]:
df["Clean.Cup"].unique()

In [ ]:
df["Clean.Cup"].describe()

En los valores de la variable Clean Cup encontramos al menos un valor mínimo de 0, que choca bastante con el resto de valores, por lo que puede que se deba a un error en la toma de datos.

Vamos a estudiar los casos que estén por debajo de la puntuación 5.

In [ ]:
df.loc[df["Clean.Cup"]<5, ["Species","Country.of.Origin","Region",	"Variety",	"Processing.Method", "Sweetness",	"Aroma",	"Flavor",	"Aftertaste",	"Acidity",	"Body",	"Quakers", "Clean.Cup"]]

Hay otros valores bajos, pero en ninguno de esos casos podemos suponer que sean errores pues la característica "Clean Cup" hace referencia a la existencia de sabores fuera de lugar en el café y es posible que haya sucedido en esas variedades.

No podemos eliminarlos ni deberíamos imputarles ningún valor pues puede que al hacerlo estemos variando información correcta.

<h4 style="text-align: center;"><ins>19 - Humedad</ins></h4>

In [ ]:
df["Moisture"].unique()

In [ ]:
df["Moisture"].describe()

Los valores estadísticos de la variable humedad parecen ser coherentes, sin valores extremos o fuera de rango; aunque consideramos poco realista la existencia de un valor de cero absoluto para una característica como la humedad de las semillas, por lo que vamos a revisarlo.

In [ ]:
df[df["Moisture"]==0].shape

Hay muchos registros con un valor de 0 en el campo Moisture, por lo que vamos a considerarlo un valor correcto y posible.

<h4 style="text-align: center;"><ins>20 - Defectos Tipo 1</ins></h4>

In [ ]:
df["Category.One.Defects"].unique()

In [ ]:
df["Category.One.Defects"].value_counts()

La mayor parte de los valores son de 0 defectos, aunque existe una gran cantidad de valores posibles. Decidimos crear agrupamientos para poder realizar mejor el análisis de esta característica.

In [ ]:
df["Cat_One_Defects_Groups"] = [str(x) if x>=0 and x<=2 else ("Between 3 and 5" if x>=3 and x<=5 else "More than 5") for x in df["Category.One.Defects"]]

<h4 style="text-align: center;"><ins>21 - Defectos Tipo 2</ins></h4>

In [ ]:
df["Category.Two.Defects"].unique()

In [ ]:
df["Category.Two.Defects"].value_counts()

Hay bastantes registros con un valor de 0 defectos, pero hay una gran cantidad de valores posibles y también decidimos crear agrupamientos para poder realizar mejor el análisis de esta característica.

In [ ]:
df["Cat_Two_Defects_Groups"] = [str(x) if x>=0 and x<=2 else ("Between 3 and 5" if x>=3 and x<=5 else "More than 5") for x in df["Category.Two.Defects"]]

<h4 style="text-align: center;"><ins>22 - Total de puntos</ins></h4>

In [ ]:
df["Total.Cup.Points"].unique()

In [ ]:
df["Total.Cup.Points"].describe()

Entre los valores estadísticos de la variable Total Cup points no parece haber valores extremos o fuera de rango.


---

Creamos un archivo pickle con los datos ya tratados.

In [ ]:
p_data_file = Path(os.path.dirname(src_path)+p_data_path+'coffe_clean.pkl')

with open(p_data_file, 'wb') as handle:
    pickle.dump(df, handle)


---

Importamos el archivo pickle creado tras las limpieza de datos para pasar a realizar el análisis univariante.

In [ ]:
p_data_file = Path(os.path.dirname(src_path)+p_data_path+'coffe_clean.pkl')

with open(p_data_file, 'rb') as handle:
    df = pickle.load(handle)

---

## ANÁLISIS UNIVARIANTE

<h4 style="text-align: center;"><ins>1 - País de Origen</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=df, y="Country.of.Origin", order= df["Country.of.Origin"].value_counts().index, ax=ax)
plt.show()

Los principales países productores de café presentes en el dataset son centroamericanos, con algunos africanos en posiciones centrales y los asiáticos con muy poca representación.

<h4 style="text-align: center;"><ins>2 - Especie</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sns.countplot(data=df, x="Species", order= df["Species"].value_counts().index, ax=ax)
plt.show()

El dataset está claramente poblado por más cantidad de registros de la especie "Arabica" que de la especie "Robusta".

Esto debe ser tenido en cuenta en el momento de la realización de predicciones, pues un desequilibrio tan grande afectará negativamente y ha de ser evitado.

<h4 style="text-align: center;"><ins>3 - Variedad</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=df, y="Variety", order= df["Variety"].value_counts().index, ax=ax)
plt.show()

La presencia de tantos registros sin un valor específico de la variedad de café cultivada nos afectará negativamente a la hora de predecir qué variedad es mejor, por lo que lo mejor sería no tener en cuenta este atributo en su momento, aunque vamos a mantenerlo durante el análisis.

<h4 style="text-align: center;"><ins>4 - Altitud</ins></h4>

En vez de graficar los valores de altitud media usaremos los grupos que hemos generado anteriormente a fin de simplificar el análisis.

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sns.countplot(data=df, x="altitude_groups", order= df["altitude_groups"].value_counts().index, ax=ax)
plt.show()

Puede verse que una inmensa mayoría de las plantaciones se encuentran entre los 1000 y los 2000m de altura.

<h4 style="text-align: center;"><ins>5 - Método de procesado</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 7))
sns.countplot(data=df, x="Processing.Method", order= df["Processing.Method"].value_counts().index, ax=ax)
plt.show()

El método de procesamiento más habitual parece ser el de Lavado, siendo el secado natural el segundo; aunque muy por debajo.

<h4 style="text-align: center;"><ins>6 - Color</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(7, 7))
sns.countplot(data=df, x="Color", order= df["Color"].value_counts().index, ax=ax)
plt.show()

Como era esperado, la mayor parte de las semillas de café presentan una coloración verde.

<h4 style="text-align: center;"><ins>7 - Quakers</ins></h4>

En vez de analizar la variable que lista todas las posibles ocurrencias, lo haremos con la versión agrupada.

In [ ]:
fig, ax = plt.subplots(figsize=(10, 7))
sns.countplot(data=df, x="quakers_groups", order= df["quakers_groups"].value_counts().index, ax=ax)
plt.show()

Prácticamente todos los sacos de café estaban carentes de semillas inmaduras y no tostadas

<h4 style="text-align: center;"><ins>8 - Año de cosecha</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=df.sort_values(by=["HarvestYear_Calc"]), x="HarvestYear_Calc", ax=ax)
plt.show()

Casi el total de las semillas corresponden a cosechas entre el 2012 y el 2016, con un pequeño grupo del que desconocemos la fecha.

<h4 style="text-align: center;"><ins>9 - Variety</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=df, y="Variety", order= df["Variety"].value_counts().index, ax=ax)
plt.show()

Tenemos demasiados registros para los que no conocemos la variedad. Eso puede ser un problema a la hora de predecir la calidad, por lo que no deberíamos tener en cuenta esos registros o buscar la forma de averiguarlo.

<h4 style="text-align: center;"><ins>10 - Aroma</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Aroma", kde=True, ax=axs[0])
sns.stripplot(x=df["Aroma"], ax=axs[1])
sns.boxplot(data=df, x="Aroma", ax=axs[2])


Las gráficas de distribución y dispersión parecen indicar que la mayor parte de los valores para el aroma se encuentran en una puntuación entre 7 y 8, con algunos valores residuales por debajo de 6.5

De acuerdo al diagrama de caja, encontramos un valor potencialmente fuera de rango con una puntuación cercana al 5, aunque la consideramos válida.

<h4 style="text-align: center;"><ins>11 - Sabor</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Flavor", kde=True, ax=axs[0])
sns.stripplot(x=df["Flavor"], ax=axs[1])
sns.boxplot(data=df, x="Flavor", ax=axs[2])

Las gráficas de distribución y dispersión muestran que los valores de la variable Flavor se encuentran en la horquilla entre el 6 y el 8, con escasos valores potencialmente fuera de rango pero válidos dentro de la puntuación.

La mediana está ligeramente desplazada hacia el lado superior del rango intercuartílico.

<h4 style="text-align: center;"><ins>12 - Regusto</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Aftertaste", kde=True, ax=axs[0])
sns.stripplot(x=df["Aftertaste"], ax=axs[1])
sns.boxplot(data=df, x="Aftertaste", ax=axs[2])

Las gráficas de distribución y dispersión muestran que los valores de la variable Aftertaste también se encuentran en la horquilla entre el 6 y el 8, con algunos casos valores potencialmente fuera de rango pero válidos dentro de la puntuación.

La mediana está muy centrada en el rango intercuartílico, lo que refuerza la idea de que sigue una distribución próxima a la normal.

<h4 style="text-align: center;"><ins>13 - Cuerpo</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Body", kde=True, ax=axs[0])
sns.stripplot(x=df["Body"], ax=axs[1])
sns.boxplot(data=df, x="Body", ax=axs[2])

De acuerdo a las gráficas de distribución y dispersión, los valores de la variable Body también se encuentran en la horquilla entre el 7 y el 8, con algunos potencialmente fuera de rango pero válidos dentro de la puntuación.

La mediana está casi centrada en el rango intercuartílico, lo que refuerza la idea de que sigue una distribución próxima a la normal.

<h4 style="text-align: center;"><ins>14 - Dulzor</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Sweetness", kde=True, ax=axs[0])
sns.stripplot(x=df["Sweetness"], ax=axs[1])
sns.boxplot(data=df, x="Sweetness", ax=axs[2])

La distribución de la variable está prácticamente centralizada en el valor 10, con un pequeño grupo en el valor 9 y el resto en 8 o inferior.

La distribución no se aproxima a la normal.

Esta característica puede que no sea de mucha ayuda en la predicción, pues la práctica totalidad de los valores son el máximo.

<h4 style="text-align: center;"><ins>15 - Acidez</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Acidity", kde=True, ax=axs[0])
sns.stripplot(x=df["Acidity"], ax=axs[1])
sns.boxplot(data=df, x="Acidity", ax=axs[2])

De acuerdo a las gráficas de dispersión y distribución podemos inferir que los datos están concentrados en la horquilla de valores entre el 7 y el 8, con algunos valores fuera de ese rango pero válidos dentro de lo esperado.

La distribución, dada la gráfica y la posición de la mediana en el diagrama de caja, se aproxima a la normal.

<h4 style="text-align: center;"><ins>16 - Equilibrio</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Balance", kde=True, ax=axs[0])
sns.stripplot(x=df["Balance"], ax=axs[1])
sns.boxplot(data=df, x="Balance", ax=axs[2])

Examinando las gráficas se observa que los valores de la variable Balance se distribuyen alrededor de las puntuaciones 7 y 8, con algunos valores fuera de ese rango, pero perfectamente válidos.

La distribución, de acuerdo a la gráfica y a la posición de la mediana en el diagrama de caja, aunque no sigue una normal sí que se aproxima a ella.

<h4 style="text-align: center;"><ins>17 - Uniformidad</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Uniformity", kde=True, ax=axs[0])
sns.stripplot(x=df["Uniformity"], ax=axs[1])
sns.boxplot(data=df, x="Uniformity", ax=axs[2])

Como ya esperábamos tras el análisis de los datos de la variable Uniformity la distribución está entre la puntuación 9 y 10, muy alejada de una distribución normal.

Al igual que con la variable Sweetness, es posible que esta variable no nos sea de mucha ayuda a la hora de realizar la predicción, dado que la mayor parte de sus valores están en el máximo.

<h4 style="text-align: center;"><ins>18 - Clean Cup</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Clean.Cup", kde=True, ax=axs[0])
sns.stripplot(x=df["Clean.Cup"], ax=axs[1])
sns.boxplot(data=df, x="Clean.Cup", ax=axs[2])

El análisis de los datos de la variable Clean Cup ya nos apuntaba que la distribución iba a estar entre la puntuación 9 y 10, muy alejada de una distribución normal. Además, tenemos la presencia de valores fuera de rango que no podemos tratar pues pueden ser correctos dada la característica.

Al igual que antes, es posible que esta variable no nos sea de mucha ayuda a la hora de realizar la predicción, dado que la mayor parte de sus valores están en el máximo.

<h4 style="text-align: center;"><ins>19 - Humedad</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Moisture", kde=True, ax=axs[0])
sns.stripplot(x=df["Moisture"], ax=axs[1])
sns.boxplot(data=df, x="Moisture", ax=axs[2])

De acuerdo a lo que podemos interpretar de las gráficas de distribución y dispersión, la mayor parte de los datos se encuentran entre los valores 0,10 y 0,15 de humedad, con un gran número de ellos también con el valor 0,00.

Seguimos pensando que el valor 0,00 de humedad es algo poco probable siendo semillas de café, pero no podemos realizar ningún cambio ni imputación.

<h4 style="text-align: center;"><ins>20 - Defectos Tipo 1</ins></h4>

Dado que en el análisis de los datos decidimos que era mejor realizar agrupamientos, este análisis univariante lo realizaremos sobre esos agrupamientos para una mayor claridad en el análisis.

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=df, x="Cat_One_Defects_Groups", ax=ax)
plt.show()

Prácticamente todos los registros tienen un valor 0 para la variable de Defectos Tipo 1.

<h4 style="text-align: center;"><ins>21 - Defectos Tipo 2</ins></h4>

In [ ]:
fig, ax = plt.subplots(figsize=(15, 10))
sns.countplot(data=df, x="Cat_Two_Defects_Groups", ax=ax)
plt.show()

Podemos ver que para esta característica hay bastantes registros con un valor de 0 defectos, siendo el siguiente en cantidad los que presentan entre 3 y 5 defectos.

<h4 style="text-align: center;"><ins>22 - Total de puntos</ins></h4>

In [ ]:
fig, axs = plt.subplots(figsize=(15, 7), ncols=3)
sns.histplot(data=df, x="Total.Cup.Points", kde=True, ax=axs[0])
sns.stripplot(x=df["Total.Cup.Points"], ax=axs[1])
sns.boxplot(data=df, x="Total.Cup.Points", ax=axs[2])